## UTMCC DataViz Module 20 Final Team Project
### Food Deserts in the Austin, Texas Metro Area

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pandas as pd
from scipy import stats
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import datetime as dt
import calendar
import random

%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import statistics
from flask import Flask, jsonify
from mpl_toolkits.mplot3d import Axes3D

# Python SQL toolkit and Object Relational Mapper
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import extract

In [3]:
# Import and read csv files
food_atlas_df = pd.read_csv("resources/food_access_research_atlas.csv")
# food_var_df = pd.read_csv("resources/food_access_variable_lookup.csv")

In [4]:
food_atlas_df.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,...,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,...,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,...,439,2576,647,17,5,11,117,87,108,120
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,...,904,4086,193,18,4,11,74,85,19,82
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,...,1126,8666,1437,296,9,48,310,355,198,488


In [ ]:
# food_var_df.head()

## Preprocessing the data

In [5]:
food_atlas_df.dtypes

CensusTract       int64
State            object
County           object
Urban             int64
POP2010           int64
                  ...  
TractAIAN         int64
TractOMultir      int64
TractHispanic     int64
TractHUNV         int64
TractSNAP         int64
Length: 147, dtype: object

In [6]:
# Create new df keeping only Texas
food_texas_df = food_atlas_df[(food_atlas_df["State"]=="Texas")]
food_texas_df

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
61468,48001950100,Texas,Anderson,0,4685,1874,0,49,0.010459,0,...,912,4012,452,22,0,13,186,236,125,218
61469,48001950401,Texas,Anderson,0,5422,77,1,5219,0.962560,0,...,24,1825,2266,21,0,5,1305,1324,5,0
61470,48001950402,Texas,Anderson,0,7535,83,1,7315,0.970803,0,...,134,2591,3248,13,0,10,1673,1737,0,0
61471,48001950500,Texas,Anderson,1,4377,1604,0,86,0.019648,1,...,627,2737,800,19,2,20,799,1389,66,288
61472,48001950600,Texas,Anderson,1,6405,2253,0,96,0.014988,1,...,791,3831,1674,68,4,43,785,1253,194,412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66721,48505950400,Texas,Zapata,0,5610,1741,0,0,0.000000,1,...,555,5264,5,12,0,30,299,5176,58,539
66722,48507950100,Texas,Zavala,0,1232,388,0,0,0.000000,1,...,125,1056,4,0,0,2,170,1104,55,164
66723,48507950200,Texas,Zavala,0,1880,590,0,0,0.000000,1,...,203,1612,21,2,9,13,223,1635,59,155
66724,48507950301,Texas,Zavala,0,2254,628,0,348,0.154392,0,...,225,2096,14,1,0,3,140,2109,160,384


In [7]:
# Create new df keeping only select Counties in the Austin Metro Area
counties = ["Bastrop", "Caldwell", "Hays", "Travis", "Williamson"]
food_austin_df = food_texas_df.loc[food_texas_df["County"].isin(counties)]
food_austin_df

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
61529,48021950100,Texas,Bastrop,0,8608,3063,0,0,0.000000,0,...,1028,6507,383,53,4,65,1596,2660,51,351
61530,48021950200,Texas,Bastrop,1,7955,2625,0,186,0.023381,1,...,891,4521,1328,32,14,76,1984,3674,148,444
61531,48021950300,Texas,Bastrop,0,12927,4734,0,71,0.005492,0,...,1468,9971,1213,156,18,88,1481,3012,95,568
61532,48021950400,Texas,Bastrop,1,7984,3127,0,456,0.057114,0,...,1199,6312,800,78,3,62,729,1711,255,261
61533,48021950501,Texas,Bastrop,0,8008,2168,0,1519,0.189685,0,...,566,5209,849,37,12,123,1778,3253,128,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66673,48491021507,Texas,Williamson,1,9196,3293,0,1,0.000109,0,...,444,6315,1227,421,15,44,1174,2710,179,277
66674,48491021508,Texas,Williamson,1,6531,2402,0,0,0.000000,0,...,301,4744,761,303,7,28,688,1627,23,133
66675,48491021601,Texas,Williamson,0,3278,1120,0,0,0.000000,1,...,236,2569,102,19,6,15,567,954,5,26
66676,48491021602,Texas,Williamson,0,2857,959,0,0,0.000000,0,...,332,2464,47,19,0,16,311,906,12,106


In [8]:
# Create new df with select feature columns
food_desert_df = food_austin_df[["LILATracts_1And10", "lasnaphalf", "lasnaphalfshare", "lahunvhalf", "lahunvhalfshare", "lasnap1", "lasnap1share", "lahunv1", "lahunv1share", "lasnap10", "lasnap10share", "lahunv10", "lahunv10share", "lasnap20", "lasnap20share",  "lahunv20", "lahunv20share"]]
food_desert_df

,LILATracts_1And10,lasnaphalf,lasnaphalfshare,lahunvhalf,lahunvhalfshare,lasnap1,lasnap1share,lahunv1,lahunv1share,lasnap10,lasnap10share,lahunv10,lahunv10share,lasnap20,lasnap20share,lahunv20,lahunv20share
61529,0,325.133846,0.106149,48.548857,0.015850,319.907937,0.104443,47.889963,0.015635,5.159378,0.001684,0.922672,0.000301,0.0,0.0,0.0,0.0
61530,1,368.584869,0.140413,120.671667,0.045970,233.501673,0.088953,81.658810,0.031108,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
61531,0,590.025451,0.124636,100.788957,0.021290,549.545092,0.116085,84.946165,0.017944,48.577263,0.010261,8.752010,0.001849,0.0,0.0,0.0,0.0
61532,0,241.111748,0.077106,191.810180,0.061340,188.984455,0.060436,114.453080,0.036602,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
61533,0,307.709167,0.141932,144.460704,0.066633,307.703411,0.141930,144.458152,0.066632,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66673,0,253.269297,0.076911,147.346423,0.044745,200.157469,0.060783,94.092539,0.028574,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
66674,0,65.461476,0.027253,3.168106,0.001319,37.486129,0.015606,0.346386,0.000144,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
66675,1,24.931507,0.022260,6.618853,0.005910,24.931507,0.022260,6.618853,0.005910,15.243361,0.013610,2.243933,0.002004,0.0,0.0,0.0,0.0
66676,0,104.153688,0.108607,13.420352,0.013994,104.153688,0.108607,13.420352,0.013994,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [ ]:
# print(food_desert_df)

## SQLite database

In [9]:
# SQLAlchemy Create Engine, to access and query the SQLite database file
engine = create_engine("sqlite:///food_desert.sqlite", echo=True)
sqlite_connection = engine.connect()

2020-12-18 19:31:10,679 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-18 19:31:10,682 INFO sqlalchemy.engine.base.Engine ()
2020-12-18 19:31:10,684 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-18 19:31:10,685 INFO sqlalchemy.engine.base.Engine ()


In [ ]:
# to reflect the database into a new model
#Base = automap_base()
# to reflect the db tables 
#Base.prepare(engine, reflect=True)

### Create the database

In [10]:
# Create the database
conn = sqlite3.connect("DB1.sqlite")
cur = conn.cursor()

In [12]:
cur.execute('CREATE TABLE Food_desert_Austin (LILATracts_1And10, lasnaphalf, lasnaphalfshare, lahunvhalf, lahunvhalfshare, lasnap1, lasnap1share, lahunv1, lahunv1share, lasnap10, lasnap10share, lahunv10, lahunv10share, lasnap20, lasnap20share,  lahunv20, lahunv20share)')
conn.commit()

In [13]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [14]:
# Data from dataframe to SQLite
food_desert_df.to_sql('Food_desert_Austin', conn, if_exists='replace')

In [15]:
cur.execute('''SELECT * FROM Food_desert_Austin''')

In [16]:
for row in cur.fetchall():
    print(row)

.07144449999998, 0.10381418199999999, 48.538971000000004, 0.019755381000000002, 1.7798392769999998, 0.000724395, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66339, 0, 113.0478145, 0.044263044, 84.69485295, 0.03316165, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66340, 1, 330.79864530000003, 0.359955, 68.19571140000001, 0.074206432, 323.9826608, 0.35253826, 66.88980403, 0.072785423, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66341, 0, 344.3517653, 0.162814073, 35.7769961, 0.016915837, 14.98002571, 0.007082754, 1.272340008, 0.000601579, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66342, 1, 394.82307799999995, 0.20362201, 209.0910897, 0.10783449699999999, 71.55946077, 0.036905343, 37.62040336, 0.019401962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66343, 0, 225.35482119999998, 0.10550319300000001, 129.7266683, 0.060733459000000004, 47.13064164, 0.022064907, 21.97454184, 0.010287707, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(66344, 0, 73.61248472, 0.060288685, 53.68982266, 0.0439720

In [ ]:
# From SQLite to DataFrame
# food_desert2_df = pd.DataFrame(cur.fetchall(), columns=['LILATracts_1And10', 'lasnaphalf', 'lasnaphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnap1', 'lasnap1share', 'lahunv1', 'lahunv1share', 'lasnap10', 'lasnap10share', 'lahunv10', 'lahunv10share', 'lasnap20', 'lasnap20share',  'lahunv20', 'lahunv20share'])
# print(food_desert2_df)

In [17]:
# Creating a new dataframe from the SQLite database table.
food_desert_Austin_df = pd.read_sql_query("SELECT * FROM Food_desert_Austin", conn)

In [18]:
food_desert_Austin_df

,index,LILATracts_1And10,lasnaphalf,lasnaphalfshare,lahunvhalf,lahunvhalfshare,lasnap1,lasnap1share,lahunv1,lahunv1share,lasnap10,lasnap10share,lahunv10,lahunv10share,lasnap20,lasnap20share,lahunv20,lahunv20share
0,61529,0,325.133846,0.106149,48.548857,0.015850,319.907937,0.104443,47.889963,0.015635,5.159378,0.001684,0.922672,0.000301,0.0,0.0,0.0,0.0
1,61530,1,368.584869,0.140413,120.671667,0.045970,233.501673,0.088953,81.658810,0.031108,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,61531,0,590.025451,0.124636,100.788957,0.021290,549.545092,0.116085,84.946165,0.017944,48.577263,0.010261,8.752010,0.001849,0.0,0.0,0.0,0.0
3,61532,0,241.111748,0.077106,191.810180,0.061340,188.984455,0.060436,114.453080,0.036602,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
4,61533,0,307.709167,0.141932,144.460704,0.066633,307.703411,0.141930,144.458152,0.066632,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,66673,0,253.269297,0.076911,147.346423,0.044745,200.157469,0.060783,94.092539,0.028574,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
346,66674,0,65.461476,0.027253,3.168106,0.001319,37.486129,0.015606,0.346386,0.000144,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
347,66675,1,24.931507,0.022260,6.618853,0.005910,24.931507,0.022260,6.618853,0.005910,15.243361,0.013610,2.243933,0.002004,0.0,0.0,0.0,0.0
348,66676,0,104.153688,0.108607,13.420352,0.013994,104.153688,0.108607,13.420352,0.013994,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [19]:
# Close the database connection
sqlite_connection.close()

## Machine Learning Model

In [28]:
# Split the preprocessed dataframe from the SQLite database into our features and target arrays
#  Remove LILATracts_1And10 target from features data
y = food_desert_Austin_df.LILATracts_1And10.values
X = food_desert_Austin_df.drop(columns="LILATracts_1And10").values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net. the number of input features and the hidden nodes for each layer.
# A good rule of thumb for a basic neural network is to have two to three times the amount of neurons in 
# the hidden layer as the number of inputs.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                540       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile and train the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
9/9 [==============================] - 0s 2ms/step - loss: 0.6112 - accuracy: 0.6947
Epoch 2/50
9/9 [==============================] - 0s 1ms/step - loss: 0.5538 - accuracy: 0.7672
Epoch 3/50
9/9 [==============================] - 0s 3ms/step - loss: 0.5094 - accuracy: 0.8321
Epoch 4/50
9/9 [==============================] - 0s 2ms/step - loss: 0.4715 - accuracy: 0.8435
Epoch 5/50
9/9 [==============================] - 0s 2ms/step - loss: 0.4394 - accuracy: 0.8626
Epoch 6/50
9/9 [==============================] - 0s 1ms/step - loss: 0.4108 - accuracy: 0.8779
Epoch 7/50
9/9 [==============================] - 0s 1ms/step - loss: 0.3872 - accuracy: 0.8740
Epoch 8/50
9/9 [==============================] - 0s 2ms/step - loss: 0.3664 - accuracy: 0.8779
Epoch 9/50
9/9 [==============================] - 0s 1ms/step - loss: 0.3494 - accuracy: 0.8817
Epoch 10/50
9/9 [==============================] - 0s 2ms/step - loss: 0.3355 - accuracy: 0.8817
Epoch 11/50
9/9 [======================

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 0.2608 - accuracy: 0.8864
Loss: 0.2607710361480713, Accuracy: 0.8863636255264282


### Create a callback to save the model's weights, and
###   Save and export the results to an HDF5 file, Hierarchical Data Format

In [25]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Export our model to HDF5 file
nn.save("food_desert_austin.h5")

## Comparisons to other ML models

In [26]:
# Create a Random Forest classifier, 64 estimators.
rf_model = RandomForestClassifier(n_estimators=64, random_state=78)
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.875


In [27]:
# Logistic Regression Model Accuracy
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)
log_classifier.fit(X_train,y_train)
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.841


## Other, Plotting and Charting 

In [ ]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotScatterMatrix(food_desert_df, 20, 10)

In [ ]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = food_desert_df
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [ ]:
plotCorrelationMatrix(food_desert_df, 24)